In [27]:
import mysql.connector
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from pandas import Timestamp
from IPython.display import display
from Functions.connection.connection import *
from Functions.AlertP1.data_cleaning import *
from Functions.AlertP1.features import *
from Functions.analysis.step_analysis import *
from Functions.AlertP1.dummy_features import *
from Functions.Models.Logistic_regression import *
from Functions.Models.evaluation import *
#from Data_with_NLP import *
import spacy
from gensim.models import Word2Vec as w2v
import re
import nltk
import gensim


creds = ["uktuserdaroglu","Vacy4201(Pri","172.20.20.4","hgo",3306]
#Connection to the database
host = creds[2]
user = creds[0]
password = creds[1]
database = creds[3]
port = creds[4]
mydb = mysql.connector.connect(host=host, user=user, database=database, port=port, password=password, auth_plugin='mysql_native_password')
mycursor = mydb.cursor()

#Safecheck to guarantee that the connection worked
mycursor.execute('SHOW TABLES;')
print(f"Tables: {mycursor.fetchall()}")
print(mydb.connection_id) #it'll give connection_id,if got connected

Tables: [('ConsultaUrgencia_doentespedidosconsultaNeurologia2012',), ('consultaneurologia2012',), ('consultaneurologia201216anon_true',), ('hgo_data_032023',)]
661


In [28]:
alertP1 = pd.read_sql("""SELECT * FROM consultaneurologia201216anon_true""",mydb)

/Users/utkuserdaroglu/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [29]:
#changing columns with date to a datetime
date_format_alertP1(alertP1)
#change the blank values in proveniencia and priority to unknown
replace_blank(alertP1)
#merge duplicated entity
#entity_duplicated(alertP1)
#changing the text to lower case
#lowering_text(alertP1)
#create a column with the output of a referral and select the refused and accepted ones
alertP1=result(alertP1)

/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/data_cleaning.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['PROVENIENCIA'][alertP1['PROVENIENCIA']=='']='unknown'
/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/data_cleaning.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['CTH_PRIOR'][alertP1['CTH_PRIOR']=='']='unknown'


In [30]:
alertP1=bef_accepted(alertP1)
class_area(alertP1)
text_length(alertP1)
referral_steps(alertP1)
speciality(alertP1)
alertP1=unit(alertP1)

/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/features.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['step'][alertP1['step']>=3]='3+'
/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/features.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['unit'][alertP1['TIPO_UNID']!='CS/USF']='HOSP'


In [31]:
#add symptoms column
#symptoms_column(alertP1)


#add exams columns
#exams(alertP1)
#add comorbidities columns
#comorbidities(alertP1)


#add medication columns
#medication_column(alertP1)



#synonyms(alertP1)
#apply lemmatization and removing stop words
#clean_text(alertP1)
df=alertP1
df.head()

,ID_DOENTE,PROCESSO,COD_REFERENCIA,COD_PZ,COD_UNID_SAUDE_PROV,UNID_PROV,TIPO_UNID,PROVENIENCIA,COD_CTH_PRIOR,CTH_PRIOR,...,resume saída,mês_saida,ano_saida,Texto,result,area,text_length,step,speciality_type,unit
0,SSHCCB,LVMVDV,LQHSOQLH,GDCADAB,3151575,USF TORRE DA MARINHA,CS/USF,CTH,3,Normal,...,12/06/2015,6,2015,"Utente de 75 anos, refere tremor desde há 5 an...",1,inside area,701,1,General Neurology,USF
1,CSAS,LVMVBD,LOHLTRTS,FZFZSDB,3152471,USF SERVIR SAUDE-CS,CS/USF,CTH,,unknown,...,02/07/2012,7,2012,Avaliação neurologica para avaliação da toma d...,1,inside area,363,1,General Neurology,USF
2,ADCFD,LVMMVJ,LQHROQHL,,0,,OUTRA,SAM,,unknown,...,13/01/2015,1,2015,cefaleias,1,outside area,9,1,Other specialities,HOSP
3,SSZDBZ,LVMMLV,LPHQULSU,CSGGHEZ,3151672,USF AMORA SAUDAVEL,CS/USF,CTH,,unknown,...,03/12/2013,12,2013,"Utente do sexo feminino com 30 anos de idade, ...",1,inside area,417,1,General Neurology,USF
4,SEAHEG,LVMNDL,LPHHVHVL,,0,,OUTRA,unknown,,unknown,...,26/02/2013,2,2013,Sem relatorio clinico,0,outside area,22,1,General Neurology,HOSP


In [32]:
full_text=df.Texto.apply(gensim.utils.simple_preprocess)
full_text

0        [utente, de, anos, refere, tremor, desde, há, ...
1        [avaliação, neurologica, para, avaliação, da, ...
2                                              [cefaleias]
3        [utente, do, sexo, feminino, com, anos, de, id...
4                                [sem, relatorio, clinico]
                               ...                        
10144                                                   []
10146                                                   []
10147                                                   []
10148                                                   []
10149                                                   []
Name: Texto, Length: 9374, dtype: object

In [33]:
model=w2v(full_text,
          min_count=3,
          window=4

)

In [36]:
print(model.wv.most_similar('parkinson'))

[('diagnóstico', 0.9966307282447815), ('doença', 0.9957289099693298), ('enxaquecas', 0.9951974153518677), ('enxaqueca', 0.9943707585334778), ('neoplasia', 0.9929117560386658), ('vive', 0.9925993084907532), ('depressão', 0.9925804734230042), ('dte', 0.9924935102462769), ('gravis', 0.9920356273651123), ('sindrome', 0.9917923808097839)]


In [ ]:
#sentence to vec